In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Flatten,Dropout,Input,Activation,Dense,GlobalAveragePooling2D,Add,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import Adam

# 모델링

In [2]:
def ensembles(index):
    result = []
    for i in range(1,index+1):
        a = pd.read_csv("./AI_models/loss_kfold_{}.csv".format(i),index_col=0)
        b = pd.read_csv("./AI_models/accuracy_kfold_{}.csv".format(i),index_col=0)
        result.append(a.values)
        result.append(b.values)
    result = np.concatenate(result,axis=1)
    ### 합쳐놓음
    
    mode_list=[]
    for j in range(len(result)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(result[j])
        for n in k:
            count_list[n]+=1
        mode = count_list.index(max(count_list))
        mode_list.append(mode)
    return np.array(mode_list).reshape(-1,1)

In [23]:
def modeling():
    inputs = Input(shape=(28,28,1))
    x = inputs
    _x = Conv2D(64,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = _x
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = Dense(10,activation='softmax')(x)
    outputs=x
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 하이퍼 파라미터

In [24]:
epochs = 2000
es = EarlyStopping(monitor='val_loss',patience=100)
reLR = ReduceLROnPlateau(patience=40,verbose=1,factor=0.5)
kfold = StratifiedKFold(n_splits=10,shuffle=True)
datagen = ImageDataGenerator(height_shift_range=(-1,1),width_shift_range=(-1,1))
datagen2 = ImageDataGenerator()
optimizer = Adam(lr=0.002,epsilon=None)

# 데이터 불러오기 및 전처리

In [25]:
df = pd.read_csv("train.csv",index_col=[0])
y = df.values[:,0].astype('int32')
x = df.values[:,2:].astype('float32')/255.0
# print(x.shape,y.shape)               # (2048, 28, 28) (2048,)
#onehot = OneHotEncoder()
#y = onehot.fit_transform(y.reshape(-1,1)).toarray().astype('float32')
x = x.reshape(-1,28,28,1)
# x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.15)
# x_train = x_train.reshape(-1,28,28,1)#[:,2:26,2:26,:]
# x_val = x_val.reshape(-1,28,28,1)#[:,2:26,2:26,:]
# print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)
print(x.shape,y.shape) 
y

(2048, 28, 28, 1) (2048,)


array([5, 0, 4, ..., 9, 0, 5])

In [ ]:
index=1
result = 0
for train_index,val_index in kfold.split(x,y):
    optimizer = Adam(lr=0.003,epsilon=None)
    modelpath = './home_models_0205/02_04_AI_val_loss_index_{}.h5'.format(index)
    modelpath2 = './home_models_0205/02_04_AI_val_accuracy_index_{}.h5'.format(index)
    cp = ModelCheckpoint(monitor = 'val_loss',filepath=modelpath,save_best_only=True,verbose=1)
    cp2 = ModelCheckpoint(monitor = 'val_accuracy',filepath=modelpath2,save_best_only=True,verbose=1)


    x_train = x[train_index]
    x_val = x[val_index]
    y_train = y[train_index]
    y_val = y[val_index]

    onehot = OneHotEncoder()
    y_train = onehot.fit_transform(y_train.reshape(-1,1)).toarray().astype('float32')
    y_val = onehot.fit_transform(y_val.reshape(-1,1)).toarray().astype('float32')

    train_generator = datagen.flow(x_train,y_train,batch_size=32)
    val_generator = datagen.flow(x_val,y_val)
    model = modeling()
    model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics=['accuracy'])
    model.fit_generator(train_generator,validation_data = val_generator,epochs=epochs,callbacks=[cp,es,reLR,cp2])

    model = load_model(modelpath)
    model2 = load_model(modelpath2)
    df = pd.read_csv("test.csv",index_col=[0])
    x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
    y_pred = model.predict(x_test)
    y_pred2 = model2.predict(x_test)
    y_pred = np.argmax(y_pred,axis=-1)
    y_pred2 = np.argmax(y_pred2,axis=-1)
    df_sub = pd.read_csv('submission.csv',index_col=0)
    df_sub['digit'] = y_pred
    df_sub.to_csv('./home_models_0205/loss_kfold_{}.csv'.format(index))
    df_sub['digit'] = y_pred2
    df_sub.to_csv('./home_models_0205/accuracy_kfold_{}.csv'.format(index))

    print(index, " 번째 학습을 완료했습니다.")
    index+=1


    
"""
0.905
"""

Epoch 1/2000
58/58 [==============================] - ETA: 0s - loss: 14.8093 - accuracy: 0.1036
Epoch 00001: val_loss improved from inf to 88138.52344, saving model to ./home_models_0205\02_04_AI_val_loss_index_1.h5

Epoch 00001: val_accuracy improved from -inf to 0.10244, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_1.h5
58/58 [==============================] - 22s 386ms/step - loss: 14.8093 - accuracy: 0.1036 - val_loss: 88138.5234 - val_accuracy: 0.1024
Epoch 2/2000
58/58 [==============================] - ETA: 0s - loss: 2.4471 - accuracy: 0.1118
Epoch 00002: val_loss improved from 88138.52344 to 612.89612, saving model to ./home_models_0205\02_04_AI_val_loss_index_1.h5

Epoch 00002: val_accuracy improved from 0.10244 to 0.10732, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_1.h5
58/58 [==============================] - 22s 373ms/step - loss: 2.4471 - accuracy: 0.1118 - val_loss: 612.8961 - val_accuracy: 0.1073
Epoch 3/2000
58/58 [===============

Epoch 20/2000
58/58 [==============================] - ETA: 0s - loss: 0.4942 - accuracy: 0.8345
Epoch 00020: val_loss did not improve from 0.89917

Epoch 00020: val_accuracy did not improve from 0.74146
58/58 [==============================] - 17s 294ms/step - loss: 0.4942 - accuracy: 0.8345 - val_loss: 0.9135 - val_accuracy: 0.7220
Epoch 21/2000
58/58 [==============================] - ETA: 0s - loss: 0.4490 - accuracy: 0.8486
Epoch 00021: val_loss improved from 0.89917 to 0.87166, saving model to ./home_models_0205\02_04_AI_val_loss_index_1.h5

Epoch 00021: val_accuracy improved from 0.74146 to 0.76098, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_1.h5
58/58 [==============================] - 49s 847ms/step - loss: 0.4490 - accuracy: 0.8486 - val_loss: 0.8717 - val_accuracy: 0.7610
Epoch 22/2000
58/58 [==============================] - ETA: 0s - loss: 0.3882 - accuracy: 0.8687
Epoch 00022: val_loss did not improve from 0.87166

Epoch 00022: val_accuracy did not imp

Epoch 43/2000
58/58 [==============================] - ETA: 0s - loss: 0.1321 - accuracy: 0.9555
Epoch 00043: val_loss did not improve from 0.69660

Epoch 00043: val_accuracy did not improve from 0.81463
58/58 [==============================] - 17s 295ms/step - loss: 0.1321 - accuracy: 0.9555 - val_loss: 1.1534 - val_accuracy: 0.8146
Epoch 44/2000
58/58 [==============================] - ETA: 0s - loss: 0.1544 - accuracy: 0.9555
Epoch 00044: val_loss did not improve from 0.69660

Epoch 00044: val_accuracy did not improve from 0.81463
58/58 [==============================] - 17s 295ms/step - loss: 0.1544 - accuracy: 0.9555 - val_loss: 1.2363 - val_accuracy: 0.7756
Epoch 45/2000
58/58 [==============================] - ETA: 0s - loss: 0.2380 - accuracy: 0.9343
Epoch 00045: val_loss did not improve from 0.69660

Epoch 00045: val_accuracy did not improve from 0.81463
58/58 [==============================] - 17s 295ms/step - loss: 0.2380 - accuracy: 0.9343 - val_loss: 1.4664 - val_accuracy:

Epoch 67/2000
58/58 [==============================] - ETA: 0s - loss: 0.3468 - accuracy: 0.9164
Epoch 00067: val_loss did not improve from 0.69660

Epoch 00067: val_accuracy did not improve from 0.85854
58/58 [==============================] - 17s 295ms/step - loss: 0.3468 - accuracy: 0.9164 - val_loss: 1.4466 - val_accuracy: 0.7659
Epoch 68/2000
58/58 [==============================] - ETA: 0s - loss: 0.1952 - accuracy: 0.9506
Epoch 00068: val_loss did not improve from 0.69660

Epoch 00068: val_accuracy did not improve from 0.85854
58/58 [==============================] - 17s 295ms/step - loss: 0.1952 - accuracy: 0.9506 - val_loss: 0.9617 - val_accuracy: 0.8341
Epoch 69/2000
58/58 [==============================] - ETA: 0s - loss: 0.0939 - accuracy: 0.9712
Epoch 00069: val_loss did not improve from 0.69660

Epoch 00069: val_accuracy did not improve from 0.85854
58/58 [==============================] - 17s 295ms/step - loss: 0.0939 - accuracy: 0.9712 - val_loss: 1.0272 - val_accuracy:

Epoch 91/2000
58/58 [==============================] - ETA: 0s - loss: 0.0971 - accuracy: 0.9788
Epoch 00091: val_loss did not improve from 0.57753

Epoch 00091: val_accuracy did not improve from 0.89268
58/58 [==============================] - 17s 295ms/step - loss: 0.0971 - accuracy: 0.9788 - val_loss: 1.5106 - val_accuracy: 0.8146
Epoch 92/2000
58/58 [==============================] - ETA: 0s - loss: 0.0525 - accuracy: 0.9816
Epoch 00092: val_loss did not improve from 0.57753

Epoch 00092: val_accuracy did not improve from 0.89268
58/58 [==============================] - 17s 295ms/step - loss: 0.0525 - accuracy: 0.9816 - val_loss: 1.1123 - val_accuracy: 0.8439
Epoch 93/2000
58/58 [==============================] - ETA: 0s - loss: 0.0451 - accuracy: 0.9886
Epoch 00093: val_loss did not improve from 0.57753

Epoch 00093: val_accuracy did not improve from 0.89268
58/58 [==============================] - 17s 295ms/step - loss: 0.0451 - accuracy: 0.9886 - val_loss: 1.2928 - val_accuracy:

...

Epoch 59/2000
58/58 [==============================] - ETA: 0s - loss: 0.1623 - accuracy: 0.9512
Epoch 00059: val_loss did not improve from 0.87949

Epoch 00059: val_accuracy did not improve from 0.80976
58/58 [==============================] - 17s 295ms/step - loss: 0.1623 - accuracy: 0.9512 - val_loss: 2.0687 - val_accuracy: 0.7073
Epoch 60/2000
58/58 [==============================] - ETA: 0s - loss: 0.1364 - accuracy: 0.9582
Epoch 00060: val_loss did not improve from 0.87949

Epoch 00060: val_accuracy did not improve from 0.80976
58/58 [==============================] - 17s 295ms/step - loss: 0.1364 - accuracy: 0.9582 - val_loss: 1.8640 - val_accuracy: 0.7805
Epoch 61/2000
58/58 [==============================] - ETA: 0s - loss: 0.2086 - accuracy: 0.9468
Epoch 00061: val_loss did not improve from 0.87949

Epoch 00061: val_accuracy improved from 0.80976 to 0.81951, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_2.h5
58/58 [==============================] - 32s 549ms/

Epoch 83/2000
58/58 [==============================] - ETA: 0s - loss: 2.3764e-04 - accuracy: 1.0000
Epoch 00083: val_loss did not improve from 0.87949

Epoch 00083: val_accuracy did not improve from 0.85366
58/58 [==============================] - 17s 295ms/step - loss: 2.3764e-04 - accuracy: 1.0000 - val_loss: 1.2052 - val_accuracy: 0.8390
Epoch 84/2000
58/58 [==============================] - ETA: 0s - loss: 2.1764e-04 - accuracy: 1.0000
Epoch 00084: val_loss did not improve from 0.87949

Epoch 00084: val_accuracy did not improve from 0.85366
58/58 [==============================] - 17s 295ms/step - loss: 2.1764e-04 - accuracy: 1.0000 - val_loss: 1.1776 - val_accuracy: 0.8439
Epoch 85/2000
58/58 [==============================] - ETA: 0s - loss: 1.7499e-04 - accuracy: 1.0000
Epoch 00085: val_loss did not improve from 0.87949

Epoch 00085: val_accuracy improved from 0.85366 to 0.85854, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_2.h5
58/58 [========================

Epoch 107/2000
58/58 [==============================] - ETA: 0s - loss: 7.1213e-05 - accuracy: 1.0000
Epoch 00107: val_loss did not improve from 0.87949

Epoch 00107: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 7.1213e-05 - accuracy: 1.0000 - val_loss: 1.1377 - val_accuracy: 0.8585
Epoch 108/2000
58/58 [==============================] - ETA: 0s - loss: 2.8914e-05 - accuracy: 1.0000
Epoch 00108: val_loss did not improve from 0.87949

Epoch 00108: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 2.8914e-05 - accuracy: 1.0000 - val_loss: 1.2158 - val_accuracy: 0.8341
Epoch 109/2000
58/58 [==============================] - ETA: 0s - loss: 2.5837e-05 - accuracy: 1.0000
Epoch 00109: val_loss did not improve from 0.87949

Epoch 00109: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 2.5837e-05 - accuracy: 1.0000 - val_l

Epoch 131/2000
58/58 [==============================] - ETA: 0s - loss: 2.3324e-05 - accuracy: 1.0000
Epoch 00131: val_loss did not improve from 0.87949

Epoch 00131: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 2.3324e-05 - accuracy: 1.0000 - val_loss: 1.2084 - val_accuracy: 0.8537
Epoch 132/2000
58/58 [==============================] - ETA: 0s - loss: 1.0268e-05 - accuracy: 1.0000
Epoch 00132: val_loss did not improve from 0.87949

Epoch 00132: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 1.0268e-05 - accuracy: 1.0000 - val_loss: 1.3143 - val_accuracy: 0.8439
Epoch 133/2000
58/58 [==============================] - ETA: 0s - loss: 2.1133e-05 - accuracy: 1.0000
Epoch 00133: val_loss did not improve from 0.87949

Epoch 00133: val_accuracy did not improve from 0.86341
58/58 [==============================] - 17s 295ms/step - loss: 2.1133e-05 - accuracy: 1.0000 - val_l

58/58 [==============================] - ETA: 0s - loss: 0.9868 - accuracy: 0.6658
Epoch 00011: val_loss improved from 1.82792 to 1.20322, saving model to ./home_models_0205\02_04_AI_val_loss_index_3.h5

Epoch 00011: val_accuracy improved from 0.39512 to 0.60488, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_3.h5
58/58 [==============================] - 48s 825ms/step - loss: 0.9868 - accuracy: 0.6658 - val_loss: 1.2032 - val_accuracy: 0.6049
Epoch 12/2000
58/58 [==============================] - ETA: 0s - loss: 0.8054 - accuracy: 0.7206
Epoch 00012: val_loss did not improve from 1.20322

Epoch 00012: val_accuracy did not improve from 0.60488
58/58 [==============================] - 17s 294ms/step - loss: 0.8054 - accuracy: 0.7206 - val_loss: 1.7879 - val_accuracy: 0.4829
Epoch 13/2000
58/58 [==============================] - ETA: 0s - loss: 0.6847 - accuracy: 0.7678
Epoch 00013: val_loss improved from 1.20322 to 1.16824, saving model to ./home_models_0205\02_04_AI_val

In [ ]:
model = load_model('./AI_models/02_04_AI_val_loss_index_3.h5')
#model = load_model('./models/02_03_imger_best_index_1.h5')
df = pd.read_csv("test.csv",index_col=[0])
x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=-1)
df_sub = pd.read_csv('submission.csv',index_col=0)
df_sub['digit'] = y_pred
df_sub.to_csv('0204_f2.csv')
y_pred

In [39]:
def make_acc_loss_csv(index):
    for i in range(1,index+1):
        model1 = load_model('./home_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('./home_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict(x_test)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./home_models/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./home_models/val_accuracy_kfold_{}.csv'.format(i))
make_acc_loss_csv(5)

In [ ]:
def make_acc_loss_csv(index):
    for i in range(1,index+1):
        model1 = load_model('./AI_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('./AI_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict(x_test)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./AI_models/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./AI_models/val_accuracy_kfold_{}.csv'.format(i))
make_acc_loss_csv(8)

In [51]:
def ensembles(index):
    accuracy=[]
    for i in range(1,index+1):
        a = pd.read_csv("./home_models/val_accuracy_kfold_{}.csv".format(i),index_col=0)
        accuracy.append(a.values)
    accuracy = np.concatenate(accuracy,axis=1)
            
    loss=[]
    for i in range(1,index+1):
        b = pd.read_csv("./home_models/val_loss_kfold_{}.csv".format(i),index_col=0)
        loss.append(b.values)        
    loss = np.concatenate(loss,axis=1)
    
    accuracy_list=[]
    for j in range(len(accuracy)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(accuracy[j])
        for n in k:
            count_list[n]+=1
        accuracy_mode = count_list.index(max(count_list))
        accuracy_list.append(accuracy_mode)
    accuracy_list = np.array(accuracy_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('./home_models/accuracy_ensembles.csv')
    
    loss_list=[]
    for j in range(len(loss)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(loss[j])
        for n in k:
            count_list[n]+=1
        loss_mode = count_list.index(max(count_list))
        loss_list.append(loss_mode)
    loss_list = np.array(loss_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('./home_models/loss_ensembles.csv')
    
    
        
    
    result = np.concatenate([accuracy,loss],axis=1)
    mode_list=[]
    for j in range(len(result)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(result[j])
        for n in k:
            count_list[n]+=1
        mode = count_list.index(max(count_list))
        mode_list.append(mode)
    mode_list = np.array(mode_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = mode_list
    df.to_csv('./home_models/both_ensembles'+'.csv')
    return 

In [52]:
ensembles(5)

In [ ]:
def modeling():
    inputs = Input(shape=(28,28,1))
    x = inputs
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = _x
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    x = Flatten()(x)
    x = Dense(2048)(x)
    x = Dense(10,activation='softmax')(x)
    outputs=x
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math
 
 
 
 
 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
 
train_dir = os.path.join('./dataset/1/images/train')
val_dir = os.path.join('./dataset/1/images/val')
 
 
 
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
 
# number of classes
K = 4

input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
 
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   
 
    
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x
 
 
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x
 
 
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 
 
 
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)
 
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()
